In [ ]:
from keras.datasets import mnist
from matplotlib import pyplot
from keras.utils import to_categorical
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.models import load_model
from keras.datasets import cifar10
from keras.callbacks import EarlyStopping
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models


In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


In [ ]:
training_data_gen = ImageDataGenerator(
    rescale=1. / 255,
    zoom_range = 0.12,
    height_shift_range = 0.12,
    width_shift_range = 0.12,
    horizontal_flip = True
)
test_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
base_model = ResNet50(
    weights='imagenet',
    input_shape=(300,300, 3),
    include_top=False)

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
from keras.layers import GlobalAveragePooling2D, Dropout

model = Sequential()

model.add(ResNet50(include_top=False, weights='imagenet', input_shape=(300,300, 3)))
model.add(GlobalAveragePooling2D())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1024, activation='relu'))
model.add(Dense(120, activation='softmax'))

model.layers[0].trainable=False
model.summary()


In [ ]:
model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(
    monitor='accuracy',
    patience=1,
    restore_best_weights=True
)
model.fit(x = X_train, y= y_train, epochs=10, callbacks=[early_stopping])

In [ ]:
result = model.evaluate(X_test, y_test)

In [ ]:
model_final = load_model('NN/model_cifar10_renet50.h5')

result = model_final.evaluate(X_test, y_test)
print(result)